<a href="https://colab.research.google.com/github/markhayes0811/geo-market/blob/main/Geo_Market_Insights2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Geo Market Insights — End-to-End (Colab)

This notebook runs the full pipeline:
1. Install requirements (Colab)
2. Generate an H3 grid over Wilmington, DE area
3. Build synthetic demand (replace with real ACS if desired)
4. Compute competition pressure from `data/inputs/competitors.csv`
5. Score and rank candidate sites
6. Render an interactive Folium map


In [17]:
# Make sure we're on the new API
!pip -q install "h3>=4.0.0"

import h3
print("h3 version:", h3.__version__)
assert hasattr(h3, "latlng_to_cell"), "h3 v4+ is required."

h3 version: 4.3.1


In [18]:
!git clone https://github.com/markhayes0811/geo-market.git /content/geo-market-insights
%cd /content/geo-market-insights

fatal: destination path '/content/geo-market-insights' already exists and is not an empty directory.
/content/geo-market-insights


In [19]:
from src.geo_utils import hexes_in_radius  # now uses v4 API

In [20]:
import sys, os
if 'google.colab' in sys.modules:
    !pip install -r requirements.txt
else:
    print('Running outside Colab; ensure requirements are installed.')

In [21]:
# If on Colab, clone or mount your repo so this path exists:
import sys, os
BASE = '/content/geo-market-insights'
if not os.path.exists(BASE):
    # Assume this notebook was opened from GitHub; clone the same repo path (edit to your username)
    !git clone https://github.com/markhayes0811/geo-market.git {BASE}
os.chdir(BASE)
print('Working dir:', os.getcwd())

Working dir: /content/geo-market-insights


In [24]:
!pip install h3==3.7.6

  Using cached h3-3.7.6.tar.gz (178 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for h3 (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for h3
Failed to build h3
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (h3)


In [25]:
import pandas as pd
import geopandas as gpd
from src.config import CENTER_LAT, CENTER_LON, RADIUS_M, H3_RESOLUTION, WEIGHT_DEMAND, WEIGHT_COMPETITION, COMPETITION_DECAY_M, SEED
from src.geo_utils import hexes_in_radius
from src.demand import synthetic_demand_for_hexes
from src.competition import load_competitors, competition_pressure_for_hexes
from src.scoring import score_sites
from src.visualize import render_map


# 1) Grid
hex_gdf = hexes_in_radius(CENTER_LAT, CENTER_LON, RADIUS_M, H3_RESOLUTION)
print('Hex count:', len(hex_gdf))

# 2) Demand
hex_dem = synthetic_demand_for_hexes(hex_gdf, seed=SEED)

# 3) Competition
competitors = load_competitors('data/inputs/competitors.csv')
hex_comp = competition_pressure_for_hexes(hex_dem, competitors, decay_m=COMPETITION_DECAY_M)

# 4) Score
hex_scored = score_sites(hex_comp, WEIGHT_DEMAND, WEIGHT_COMPETITION)
hex_scored.head()

AttributeError: module 'h3' has no attribute 'geo_to_h3'

In [23]:
import os
os.makedirs('data/outputs', exist_ok=True)
hex_scored[['h3','demand','competition','site_score']].to_csv('data/outputs/candidates_ranked.csv', index=False)
out_map = render_map(hex_scored, competitors, out_path_html='data/outputs/market_map.html', top_n=15)
out_map

NameError: name 'hex_scored' is not defined

### Next Steps
- Replace synthetic demand with **real ACS data** (median income, pop density, households) and re-score.
- Add **existing store list** separately and penalize self-cannibalization more than competitors.
- Switch to **travel-time** distances (OSRM/Valhalla) for more realism.
